In [13]:
import pandas as pd
import openpyxl
import xarray as xr
import numpy as np
import yaml

In [11]:
# Load the Excel file
file_path = "../input_files/241212_GP_Blackbird_performance_matrix.xlsx"
xls = pd.ExcelFile(file_path)

# Load the performance matrix sheet
df_perf = xls.parse('PerfMatrix_from_Sunwise')

# Reshape into long format
df = df_perf.melt(id_vars='azimuth', var_name='zenith', value_name='efficiency')


# Convert solar zenith to solar altitude
df["altitude"] = 90 - df["zenith"]

# Only non-zero entries from hourly data are relevant
df = df[df["efficiency"] > 0.0]

# Efficiency in %, avoids dropping significant places in round(1) later
df["efficiency"] *= 100

# Reduce noise in solar position by rounding each position to 2°
# Creates a lot of duplicates
base = 2

df[["azimuth", "altitude"]] = (df[["azimuth", "altitude"]] / base).round(0) * base

# Drop duplicates: Keep only highest efficiency per position
df = df.groupby(["altitude", "azimuth"]).max()

# Drop all columns except for efficiency
df = df[["efficiency"]]

da = df.to_xarray()["efficiency"]

# Interpolate values to a finer grid and fill missing values by extrapolation
# Order is relevant: Start with Azimuth (where we have sufficient values) and then continue with altitude
da = (
    da.interpolate_na("azimuth")
    .interpolate_na("altitude")
    .interpolate_na("azimuth", fill_value="extrapolate")
)

# Use rolling horizon to smooth values, average over 3x3 adjacent values per pixel
da = (
    da.rolling(azimuth=3, altitude=3)
    .mean()
    .interpolate_na("altitude", fill_value="extrapolate")
    .interpolate_na("azimuth", fill_value="extrapolate")
)

# Create second dataset, mirrored around 90° (covering 90° to -90° with same numbers)
da = da.sel(azimuth=slice(90, 270))
da_new = da.assign_coords(azimuth=(((da.azimuth - 180) * (-1)) % 360)).sortby("azimuth")
da = xr.merge([da, da_new])["efficiency"]

# Reduce resolution to coarser grid for reduced storage footprint
interpolation_step = 5
da = da.interp(
    azimuth=np.arange(da["azimuth"].min(), da["azimuth"].max(), interpolation_step),
    altitude=np.arange(0, 91, interpolation_step),
    kwargs={"fill_value": "extrapolate"},
).clip(min=0.0)

In [15]:
df = da.to_dataframe().reset_index()
df = df.rename(columns={"efficiency": "value"})
df[["altitude", "azimuth"]] = df[["altitude", "azimuth"]].astype(int)
df = df.to_dict("dict")

config = {
    "name": "Parabolic Trough field",
    "source": "Glasspoint configuration with NREL System Advisor Model",
    "technology": "parabolic trough",
    "r_irradiance": 950,  # W/m2,
    "efficiency": df,
}

with open("Glasspoint_parabolic_trough.yaml", "w") as f:
    yaml.safe_dump(config, f, default_flow_style=False, sort_keys=False)